In [1]:
import numpy as np
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1550442640350_0013,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import isnan, when, count, col

VBox()

In [3]:
sc = pyspark.SparkContext.getOrCreate()
ss = pyspark.sql.SparkSession.builder.getOrCreate()

VBox()

In [4]:
s3_address = "s3a://msds630-kaggle-competition/"
dataset_addr_book = {}
dataset_name_list = ["events", "messages", "attributes", "sessions"]
for name in dataset_name_list:
    dataset_addr_book[name] = "".join([s3_address, name, ".csv"])

dataset_addr_book

VBox()

{'attributes': 's3a://msds630-kaggle-competition/attributes.csv', 'messages': 's3a://msds630-kaggle-competition/messages.csv', 'events': 's3a://msds630-kaggle-competition/events.csv', 'sessions': 's3a://msds630-kaggle-competition/sessions.csv'}

In [5]:
attributes = sc.textFile(dataset_addr_book["attributes"]).map(lambda line : line.encode('ascii', 'ignore'))

VBox()

In [6]:
header = attributes.first()
header = header.split(",")
header

VBox()

['app_id', 'session_id', 'attribute', 'attribute_value', 'user_id_hash']

In [7]:
attributes = attributes.filter(lambda x: 'app_id' not in x)

VBox()

In [8]:
attributes = attributes.map(lambda x: x.split(','))

VBox()

In [9]:
def splitter(line):
    line = line [1:]
    if len(line)!=4: 
        line = line[:2]+[" ".join(line[2:-1])]+[line[-1]]
    line = [ l if l is not "" else None for l in line ]
    return line  

VBox()

In [10]:
attributes = attributes.map(lambda x: splitter(x))

VBox()

In [13]:
df_attributes = ss.createDataFrame(attributes)

VBox()

In [14]:
df_attributes = df_attributes.withColumnRenamed("_1", "session_id")\
                             .withColumnRenamed("_2", "attribute")\
                             .withColumnRenamed("_3", "attribute_value")\
                             .withColumnRenamed("_4", "user_id_hash")

VBox()

In [15]:
df_attributes.printSchema()

VBox()

root
 |-- session_id: string (nullable = true)
 |-- attribute: string (nullable = true)
 |-- attribute_value: string (nullable = true)
 |-- user_id_hash: string (nullable = true)

In [16]:
df_attributes.show(5)

VBox()

+-------------------+---------+--------------------+--------------------+
|         session_id|attribute|     attribute_value|        user_id_hash|
+-------------------+---------+--------------------+--------------------+
|2201961907282901522|        0|                   1|9943447915df3a45f...|
|2201961907282901522|        1|                   1|9943447915df3a45f...|
|2201961907282901522|        3|                   0|9943447915df3a45f...|
|2201961907282901522|        4|                0.00|9943447915df3a45f...|
|2201961907282901522|        5|romance_the_royal...|9943447915df3a45f...|
+-------------------+---------+--------------------+--------------------+
only showing top 5 rows

In [59]:
df_attributes.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_attributes.columns]).show()

VBox()

+----------+---------+---------------+------------+
|session_id|attribute|attribute_value|user_id_hash|
+----------+---------+---------------+------------+
|         0|        0|        8479392|           0|
+----------+---------+---------------+------------+

In [57]:
df_attributes.select('session_id').distinct().show()

VBox()

+-------------------+
|         session_id|
+-------------------+
|5558845121177764917|
| 872646361392044013|
|8215533242151202207|
|5203599648320615794|
|6988916708190298784|
|3792756332066870990|
|3180502421788295067|
|5952692610146903770|
|1374192068847361138|
|7120081605675352736|
|5877793167289165002|
|5404284609527022050|
| 106225530710586831|
|7237881887928244181|
|3939621683954022370|
|1948082813038983456|
|7326749428396481096|
|4047622568897823192|
|6840108493883168098|
|2039719602745596873|
+-------------------+
only showing top 20 rows

In [60]:
df_attributes.select('attribute').distinct().show()

VBox()

+---------+
|attribute|
+---------+
|        7|
|       15|
|       11|
|        3|
|        8|
|       16|
|       52|
|        0|
|       96|
|        5|
|       18|
|       75|
|       17|
|       78|
|       77|
|        6|
|       19|
|       48|
|       67|
|        9|
+---------+
only showing top 20 rows

In [61]:
df_attributes.select('user_id_hash').distinct().show()

VBox()

+--------------------+
|        user_id_hash|
+--------------------+
|d5804698b71c91111...|
|470195b52fda2a7bf...|
|91f039e1e45d73841...|
|1acd4823fae34d820...|
|8fc3650198d4fa9d4...|
|71f0ebc9b9c7b0747...|
|fc22162c2c4ea6d0a...|
|d16c4e76046f966b5...|
|a58f070d7a4e65737...|
|c56b5758c8c74d134...|
|b04f51610a316b11c...|
|3b1be5c17d58562da...|
|0d34cabba3d807b01...|
|8bb2578a85a90dc06...|
|1b969065197d94789...|
|cd80163bf0bedc185...|
|48401aa9b427eaaf9...|
|8673e87b5932797b4...|
|7de02f3444a8c10b6...|
|56be7c318f85a9272...|
+--------------------+
only showing top 20 rows

In [17]:
df_attributes.select('session_id').distinct().count()

VBox()

6510494

In [18]:
df_attributes.select('attribute').distinct().count()

VBox()

30

In [19]:
df_attributes.select('user_id_hash').distinct().count()

VBox()

621460